In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import skimage
from skimage import io, feature, measure, transform
import tensorflow as tf
from pandas import Series,DataFrame
import sampling as smp

In [2]:
def reduce(image, scale=4):
    reducedimage = transform.pyramid_reduce(image, downscale=scale)
    return reducedimage

In [3]:
training = pd.read_msgpack('training.bin')
non_empty_training = training[~training.maskC.isnull()]
non_empty_training.index=np.arange(len(non_empty_training))
centers = non_empty_training.maskC.as_matrix()
center_list=np.array([tuple(c) for c in centers])

In [4]:
training_centers=center_list[0:2000]

In [5]:
#### crop image to 80 x 136 to make the numbers rounder for the nn..
def build_training_data(num_images):
    data=[]
    for i in range(num_images):
        if i%100 ==0:
            print ('Reading patch ' +str(i))
        ultra_image = smp.image_pair(non_empty_training.ix[i].subject,non_empty_training.ix[i].img).image
        data.append(reduce(ultra_image)[25:,:136].flatten())
    return np.array(data)

In [6]:
time X=build_training_data(1600)

Reading patch 0
Reading patch 100
Reading patch 200
Reading patch 300
Reading patch 400
Reading patch 500
Reading patch 600
Reading patch 700
Reading patch 800
Reading patch 900
Reading patch 1000
Reading patch 1100
Reading patch 1200
Reading patch 1300
Reading patch 1400
Reading patch 1500
CPU times: user 1min 46s, sys: 8.63 s, total: 1min 55s
Wall time: 2min 8s


In [7]:
sess=tf.InteractiveSession()
### use 4x reduced images as input features
x= tf.placeholder(tf.float32,shape=[None, 10880])
x_image = tf.reshape(x, [-1,80,136,1])
y_ = tf.placeholder(tf.float32, shape=[None, 2])

In [8]:
X[0].shape

(10880,)

In [9]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [10]:
### helper functions to build Conv and MaxPool layers

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool(x,psize):
  return tf.nn.max_pool(x, ksize=[1, psize, psize, 1],
                        strides=[1, psize, psize, 1], padding='SAME')

In [11]:
### weights and biases for the first Conv layer; 5x5 filter and 32 feat.
W_conv1 = weight_variable([7, 7, 1, 32])
b_conv1 = bias_variable([32])

In [12]:
### build first Conv and MaxPool layers
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool(h_conv1,3)

In [13]:
h_pool1.get_shape()

TensorShape([Dimension(None), Dimension(27), Dimension(46), Dimension(32)])

In [14]:
### stack a second Conv + MaxPool layer; again with 32 feat.
W_conv2 = weight_variable([5, 5, 32, 32])
b_conv2 = bias_variable([32])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool(h_conv2,3)


In [15]:
h_pool2.get_shape()

TensorShape([Dimension(None), Dimension(9), Dimension(16), Dimension(32)])

In [16]:
### stack a third Conv + MaxPool layer; this time 64 feat.
W_conv3 = weight_variable([5, 5, 32, 64])
b_conv3 = bias_variable([64])

h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool(h_conv3,2)


In [17]:
h_pool3.get_shape()

TensorShape([Dimension(None), Dimension(5), Dimension(8), Dimension(64)])

In [19]:
#### add fully-connected layer

W_fc1 = weight_variable([5 * 8 * 64, 256])
b_fc1 = bias_variable([256])

h_pool3_flat = tf.reshape(h_pool3, [-1, 5*8*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)

In [20]:
### dropout

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [21]:
### output layer!
W_fc2 = weight_variable([256, 2])
b_fc2 = bias_variable([2])

y_conv=(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [22]:
### use least squares cost function
cost_fn = tf.reduce_mean(tf.square(y_conv - y_))

In [23]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cost_fn)

In [24]:
accuracy =  tf.reduce_mean(tf.square(y_conv - y_))

In [25]:
init = tf.initialize_all_variables()
sess.run(init)
saver = tf.train.Saver()

In [26]:
for i in range(10000):
    pts = np.random.randint(0,1000,size=50)
    batch = (X[pts].flatten().reshape(50,10880), training_centers[pts].flatten().reshape(50,2))
    if i%10 ==0:
        train_error = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
        print ('RMS error on training batch'+str(i)+ ':'+str(train_error))
    train_step.run(feed_dict={x:batch[0],y_:batch[1],keep_prob: 0.5})


RMS error on training batch0:72167.4
RMS error on training batch10:63512.8
RMS error on training batch20:56288.7
RMS error on training batch30:34714.7
RMS error on training batch40:20076.5
RMS error on training batch50:3036.51
RMS error on training batch60:3580.66
RMS error on training batch70:2533.92
RMS error on training batch80:2053.8
RMS error on training batch90:2157.25
RMS error on training batch100:2535.3
RMS error on training batch110:2544.65
RMS error on training batch120:2386.98
RMS error on training batch130:1948.33
RMS error on training batch140:2315.38
RMS error on training batch150:2656.56
RMS error on training batch160:2127.85
RMS error on training batch170:2350.73
RMS error on training batch180:2098.02
RMS error on training batch190:2099.58
RMS error on training batch200:2736.4
RMS error on training batch210:2277.56
RMS error on training batch220:1758.31
RMS error on training batch230:2373.1
RMS error on training batch240:1798.25
RMS error on training batch250:2087.76
R

KeyboardInterrupt: 

In [27]:
def build_test_data(num_images):
    data=[]
    for i in range(2000,2300):
        #print(i)
        if i%100 ==0:
            print ('Reading patch '+str(i))
        ultra_image = smp.image_pair(non_empty_training.ix[i].subject,non_empty_training.ix[i].img).image
        data.append(reduce(ultra_image)[25:,:136].flatten())
    return np.array(data)

In [34]:
saved_model  = saver.save(sess, "/Users/gus/CDIPS/uns/centers")

In [28]:
time x_test=build_test_data(300)

Reading patch 2000
Reading patch 2100
Reading patch 2200
CPU times: user 21.1 s, sys: 1.93 s, total: 23.1 s
Wall time: 26.2 s


In [29]:
x_test.shape

(300, 10880)

In [30]:
test_centers=center_list[2000:2300]

In [31]:
test_centers.shape

(300, 2)

In [32]:
### looks like we're overfitting:

In [33]:
print("test rms error %g"%accuracy.eval(feed_dict={
    x: x_test, y_: test_centers, keep_prob: 1.0}))

test rms error 1351.45
